In [13]:
import os
import cv2
import numpy as np
import pandas as pd

import os
import cv2
import collections
import time 
import tqdm
from PIL import Image
from functools import partial
train_on_gpu = True

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import torchvision
import torchvision.transforms as transforms
import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR

import albumentations as albu

from catalyst.data import Augmentor
from catalyst.dl import utils
from catalyst.data.reader import ImageReader, ScalarReader, ReaderCompose, LambdaReader
from catalyst.dl.runner import SupervisedRunner
from catalyst.contrib.models.segmentation import Unet
from catalyst.dl.callbacks import DiceCallback, EarlyStoppingCallback, InferCallback, CheckpointCallback

import segmentation_models_pytorch as smp

In [14]:
def visualize(image, mask, original_image=None, original_mask=None, gray=True):
    fontsize = 14
    class_dict = {0: "Fish", 1: "Flower", 2: "Gravel", 3: "Sugar"}    
    if original_image is None and original_mask is None:
        f, ax = plt.subplots(1, 5, figsize=(24, 24))

        if gray:
            ax[0].imshow(image, cmap='gray')
        else:    
            ax[0].imshow(image)
        for i in range(4):
            ax[i + 1].imshow(mask[:, :, i])
            ax[i + 1].set_title(f"Mask {class_dict[i]}", fontsize=fontsize)
    else:
        f, ax = plt.subplots(2, 5, figsize=(24, 12))

        ax[0, 0].imshow(original_image)
        ax[0, 0].set_title("Original image", fontsize=fontsize)

        for i in range(4):
            ax[0, i + 1].imshow(original_mask[:, :, i])
            ax[0, i + 1].set_title(f"Original mask {class_dict[i]}", fontsize=fontsize)

        ax[1, 0].imshow(image)
        ax[1, 0].set_title("Transformed image", fontsize=fontsize)

        for i in range(4):
            ax[1, i + 1].imshow(mask[:, :, i])
            ax[1, i + 1].set_title(
                f"Transformed mask {class_dict[i]}", fontsize=fontsize
            )

In [15]:
def get_img(image_path, gray=False):
    img = cv2.imread(image_path)
    if gray:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = img[:,:,None]
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img 

def rle_decode(mask_rle: str = "", shape: tuple = (1400, 2100)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape, order="F")

def make_mask(df: pd.DataFrame, image_name: str='img.jpg', shape: tuple = (1400, 2100)):
    encoded_masks = df.loc[df['im_id'] == image_name, 'EncodedPixels']
    masks = np.zeros((shape[0], shape[1], 4), dtype=np.float32)
    for idx, label in enumerate(encoded_masks.values):
        if label is not np.nan:
            mask = rle_decode(label)
            masks[:, :, idx] = mask         
    return masks

In [16]:
N_FOLDS = 10
SEED = 42
MODEL_NO = 0
path = "../input/"
train = pd.read_csv(f"{path}/train.csv")
train["label"] = train["Image_Label"].apply(lambda x: x.split("_")[1])
train["im_id"] = train["Image_Label"].apply(lambda x: x.split("_")[0])

In [17]:
TRAIN = '../input/train_images_525/train_images_525/'
TRAIN_C = '../input/train_images_480/'
TRAIN_C_M = '../input/train_masks_480/'

In [18]:


if not os.path.exists(TRAIN_C):
    os.makedirs(TRAIN_C)
    
if not os.path.exists(TRAIN_C_M):
    os.makedirs(TRAIN_C_M)
    
from tqdm import tqdm 

# Resize and save train images and masks
tfms = albu.Compose([albu.Resize(320, 480)])
bar = tqdm(os.listdir(TRAIN), postfix={"file":"none"})

for file in bar:
    bar.set_postfix(ordered_dict={"file":file})    
    path = os.path.join(TRAIN, file)
    img = get_img(path)    
    mask = make_mask(train, file) 
    tfmed = tfms(image=img, mask=mask)
    img = tfmed['image']
    mask = tfmed['mask']
    np.save(os.path.join(TRAIN_C_M, file), mask)
    np.save(os.path.join(TRAIN_C, file), img) 

100%|██████████| 5546/5546 [08:43<00:00, 10.59it/s, file=bbde96d.jpg]


# Test on new data

In [8]:
def get_img(fname, folder="../input/train_images_525/train_images_525", npy=False):
    if npy:
        return np.load(os.path.join(folder, fname+'.npy'))
    img = cv2.imread(os.path.join(folder, fname))
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

img = get_img('bbde96d.jpg', TRAIN_C, True)
print(type(img))
print(img.shape)

<class 'numpy.ndarray'>
(350, 525, 3)


In [9]:
import os
import cv2
import time
import tqdm
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader,Dataset
import albumentations as albu
from utils import get_mask
from augmentations import get_training_augmentation, get_validation_augmentation, get_preprocessing
"""
DataLoader. I have preprocessed and saved masks as images. Also, resized and
saved the images. I used someone else's code to decode RLE into masks.
"""

class SegmentationDataset(Dataset):
    def __init__(self, ids, transforms, preprocessing=False,
            img_db=TRAIN_C,
            mask_db=TRAIN_C_M, npy=False):
        self.ids = ids
        self.transforms = transforms
        self.preprocessing = preprocessing
        self.img_db = img_db
        self.mask_db = mask_db
        self.npy = npy

    def __getitem__(self, idx):
        id = self.ids[idx]
        image = get_img(id, self.img_db, self.npy)
        mask = get_mask(id, self.mask_db)
        augmented = self.transforms(image=image, mask=mask)
        image = augmented['image']
        mask = augmented['mask']
        if self.preprocessing:
            pre = self.preprocessing(image=image, mask=mask)
            image = pre['image']
            mask = pre['mask']

        return image, mask

    def __len__(self):
        return len(self.ids)

    def __repr__(self):
        return "Training dataset for Segmentation task. Returns [img, mask(s)]."


class SegmentationDataset_withid(Dataset):
    def __init__(self, ids, transforms, preprocessing=False,
            img_db="input/train_images_525/train_images_525",
            mask_db="input/mask"):
        self.ids = ids
        self.transforms = transforms
        self.preprocessing = preprocessing
        self.img_db = img_db
        self.mask_db = mask_db

    def __getitem__(self, idx):
        id = self.ids[idx]
        image = get_img(id, self.img_db)
        mask = get_mask(id, self.mask_db)
        augmented = self.transforms(image=image, mask=mask)
        image = augmented['image']
        mask = augmented['mask']
        if self.preprocessing:
            pre = self.preprocessing(image=image, mask=mask)
            image = pre['image']
            mask = pre['mask']

        return image, mask, id

    def __len__(self):
        return len(self.ids)

    def __repr__(self):
        return "Training dataset for Segmentation task. Returns [img, mask(s)]."


In [11]:
identity = lambda x: x
train_ids = os.listdir(TRAIN)
dataset = SegmentationDataset(ids=train_ids,
                    transforms=get_training_augmentation(),
                    preprocessing=None,
                    img_db=TRAIN_C,
                    mask_db=TRAIN_C_M, npy=True)

In [12]:
for x in dataset:
    print(x[0].shape)
    print(type(x[0]))
    print(x[1].shape)
    print(type(x[1]))
    break

(350, 525, 3)
<class 'numpy.ndarray'>
(350, 525, 4)
<class 'numpy.ndarray'>
